# Homework 1
 
## Group 10

## 1. Understanding MDPs

### 1.1 Chess

- States S: All conceivable legal game states.
- Actions A: All legal moves.
- Probabilistic state dynamics: Deterministic for the game of chess. We can be certain that if we take an action we always end up in the state we want.
- Reward dynamics: The reward you get changes based on the value of the piece you capture. 
- Initial state $p_0$: All pieces positioned in their beginning state. No action has been taken.
- Policy π: The policy describes what legal move A to take for each game state S

\\

### 1.2 LunarLander

- States S: 8-dimensional vector with x and y being the coordinates of the lander, its linear velocity in x and y, its angle, its angular velocity, two booleans that represent whether each leg is connected to the ground.
- Actions A: Do nothing, fire left orientation engine, fire right orientation engine, fire the main engine.
- Probabilistic state dynamics: Dependent on the physics simulation of the environment.
- Reward dynamics: Crashing (-100), Coming to rest (+100), Each leg with ground contact (+10), firing main engine (-0.3), Firing side engine (-0.03), Solved (+200).
- Initial state: The lander is located at the top center.
- Policy π: The policy describes what move A to take for each game state S.

\\

### 1.3 Model Based RL: Accessing Environment Dynamics

Policy evaluation evaluates the current policy and computes the Value function $V(s) = \pi(a|s) \sum_s',r [p(s', r|s,a)(r+γV_π(s'))]]$

Polivy Iteration updates the  policy based on the result of the policy evaluation, by updating the optimal action to take for each state s, based on its value.

\\

The environment dynamics is a combination of reward function and state transition dynamics. Based on the action a that is taken in the state s, the environment dynamics define the reward and the state s' we end up in. For instance, an agent would gain a certain reward by collecting the points and reaching to the end of the respective level in the game of Super Mario. Also taking action a leads to state s' based on the velocity and the current position of mario. Another example would be that of the robotic arm trying to manipulate or move a certain object. The agent would aquire some rewards for sucessfully moving the object and the state transition dynamics would be described by real world physics. 

\\

The true environment dynamics are not always known. When considering the Mario example again. Even if we have all the visual information of the game and we know exactly how the character moves (in terms of physics) our moves could still be obstructed by hidden blocks. Also the reward function is defined by us in an arbitrary fashion. Even if we are not certain about the true underlying environment dynamics we can still try and approximate reality and use the approximation for our training.

## 2. Implementing a GridWorld

### 2.1 Look up some examples

1. Classic Grid World
https://notebook.community/spro/practical-pytorch/reinforce-gridworld/reinforce-gridworld

2. Windy Grid World
https://wocken-datenliebe.com/2019/06/21/windy-gridworld-rl/

3. Multi-Agent Grid World
https://github.com/opocaj92/GridWorldEnvs

### 2.2 Implementing the MDP

- The MDP uses a stochastic policy 
- The two complexities included are a trap at tile 4,4 and a blocked tile at 2,2

In [1]:
import numpy as np
import random as rd

In [30]:
LEFT = 0
TOP = 1
RIGHT = 2
DOWN = 3

class GridWorld():
    def __init__(self, size=(5,6)):
        self.size = size
        self.board = np.zeros(size)
        self.board[4,3] = 1
        self.board[4,4] = -1
        self.state = (0,0)

    def get_size(self):
        return self.size

    def get_state(self):
        return self.state

    def get_reward(self):
        return self.board[self.state]

    def is_legal(self, next_state):
        if ((next_state[0] >= 0) and (next_state[0] < len(self.board))):
            if ((next_state[1] >= 0) and (next_state[1] < len(self.board[0]))):
                if(next_state != (2,2)):
                    return True 
                else: 
                    return False
    
    def is_goal(self):
        if (self.board[self.state] == -1) or (self.board[self.state] == 1):
            return True
        else: 
            return False

    def move(self, action): 
        # origin at top left
        # TOP
        if action == TOP: 
            next_state = (self.state[0] - 1, self.state[1])
        # RIGHT
        if action == RIGHT:
            next_state = (self.state[0], self.state[1] + 1)
        # DOWN
        if action == DOWN:
            next_state = (self.state[0] + 1, self.state[1])
        # LEFT
        if action == LEFT:
            next_state = (self.state[0], self.state[1] -1)
        # check if move is legal
        if self.is_legal(next_state):
            self.state = next_state

    def reset(self):
        self.state = (0,0)

## 3. Implementing a policy 

### 3.1 Implementing the basic agent

In [31]:
class Agent():
    def __init__(self):
        self.grid_world = GridWorld()
        self.actions = [TOP, RIGHT, DOWN, LEFT]
    
    def get_action(self):
        r = rd.random()
        if(r < 0.8):
            return rd.randint(2,3)
        if(r >= 0.8):
            return rd.randint(0,1)

    def walk(self):
        episode = []
        self.grid_world.reset()
        while(not self.grid_world.is_goal()):
            state = self.grid_world.get_state()
            action = self.get_action()
            self.grid_world.move(action) 
            reward = self.grid_world.get_reward()
            episode.append((state, action, reward))
        return episode
        

### 3.2 Evaluate the policy

In [32]:
agent = Agent()
gamma = 0.9

V = {}
N = {}
for i in range(agent.grid_world.get_size()[0]):
    for j in range(agent.grid_world.get_size()[1]):
        s = (i, j)
        V[s] = 0
        N[s] = 0

for ep in range(1000):
    episode = agent.walk()
    G = 0
    visited = []
    for i, (s, a, r) in enumerate(episode[::-1]):
        if (s == (4,3)):
            print("h")
        G = gamma * G + r
        if s not in visited:
            visited.append(s)
            N[s] += 1
            V[s] += (G - V[s]) / N[s]

for i in range(agent.grid_world.get_size()[0]):
    for j in range(agent.grid_world.get_size()[1]):
        s = (i,j)
        print("V({}) = {}".format(s, V[s]))

V((0, 0)) = 0.08165910264121884
V((0, 1)) = 0.01018289956785814
V((0, 2)) = -0.07141164490926206
V((0, 3)) = -0.1287400542778063
V((0, 4)) = -0.18070942640115667
V((0, 5)) = -0.20953826176838725
V((1, 0)) = 0.1769897251345133
V((1, 1)) = 0.10483687358627423
V((1, 2)) = -0.05218940480790797
V((1, 3)) = -0.10580964523297216
V((1, 4)) = -0.24423254608270828
V((1, 5)) = -0.28080171398178194
V((2, 0)) = 0.2896261493663063
V((2, 1)) = 0.33608438178639605
V((2, 2)) = 0
V((2, 3)) = -0.04402424200270676
V((2, 4)) = -0.3620308220617057
V((2, 5)) = -0.38597031624824113
V((3, 0)) = 0.3876444435072291
V((3, 1)) = 0.4808409389353765
V((3, 2)) = 0.47959218539631326
V((3, 3)) = 0.24028481278011565
V((3, 4)) = -0.6211441501602045
V((3, 5)) = -0.6337651924860249
V((4, 0)) = 0.5211567736007382
V((4, 1)) = 0.6923447274594741
V((4, 2)) = 0.9071323119998644
V((4, 3)) = 0
V((4, 4)) = 0
V((4, 5)) = -0.9341415098553923
